In [1]:
# code base is from below
# https://github.com/keras-team/keras-io/blob/cbda610b5a73b517aa750e1ff14d7c7a70aae91e/examples/vision/ipynb/retinanet.ipynb

In [2]:
import os
import re
import zipfile

import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

import sys
sys.path.append('../')
from encode import LabelEncoder
from model import *
from losses import RetinaNetLoss

from feed import train_data_loader, val_data_loader
from inference import DecodePredictions
from box_utils import visualize_detections
from preprocess import resize_and_pad_image, preprocess_data
print(tf.__version__)

2.4.1


In [3]:
# !pip install tensorflow-gpu==2.4

In [4]:
# !pip list | grep tensorflow
# !pip uninstall tensorflow-gpu -y


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)
#     tensorflow gpu 2.5.0 version에서 gpu 인식 실패, 2.4.0으로 해야 된다.
print(tf.test.is_gpu_available()   )
tf.config.list_physical_devices('GPU')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# url = "https://github.com/srihari-humbarwadi/datasets/releases/download/v0.1.0/data.zip"
# filename = os.path.join(os.getcwd(), "../data.zip")
# keras.utils.get_file(filename, url)


# with zipfile.ZipFile("../data.zip", "r") as z_fp:
#     z_fp.extractall("../")

In [7]:
model_dir = "retinanet/"
label_encoder = LabelEncoder()

num_classes = 80
batch_size = 2

learning_rates = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values=learning_rates
)


In [8]:
resnet50_backbone = get_resnet_backbone()
loss_fn = RetinaNetLoss(num_classes)
model = RetinaNet(num_classes, resnet50_backbone)

optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
model.compile(loss=loss_fn, optimizer=optimizer)
print(model.trainable_variables[0][0][0][0]) 

tf.Tensor(
[ 0.02825263 -0.01187372  0.00151489  0.0174512   0.02267267 -0.07706022
  0.02593261  0.00758947  0.01483854  0.00269824 -0.02794643  0.01281348
 -0.00239688 -0.02064885 -0.00565647 -0.01334927 -0.07256436 -0.04735989
  0.02042213  0.06503097 -0.00141359  0.0067685  -0.00452705  0.00789207
  0.0037927  -0.02793706 -0.02059007 -0.01074472 -0.03362909 -0.00028938
 -0.01099383  0.00777401 -0.02380833  0.00196355  0.04691625  0.00554846
  0.01841898 -0.01014032 -0.01754054  0.09017435  0.02368956 -0.02602403
  0.01441176 -0.01233377 -0.01345379 -0.00706803 -0.02015607 -0.012672
 -0.0035574   0.00865486  0.07378456  0.0110056   0.03461925  0.01144895
  0.001062    0.00103144  0.00486954  0.02316773  0.0355238  -0.00459298
 -0.00914957 -0.0107004  -0.05279828 -0.00136667], shape=(64,), dtype=float32)


In [9]:
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="loss",
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
    )
]

In [10]:
#  set `data_dir=None` to load the complete dataset

(train_dataset, val_dataset), dataset_info = tfds.load(
    "coco/2017", split=["train", "validation"], with_info=True, data_dir="../data"
)
print(type(train_dataset))
print(len(train_dataset))
print(len(val_dataset))

train_loader = train_data_loader(train_dataset, label_encoder, batch_size=1)
val_loader = train_data_loader(val_dataset, label_encoder, batch_size=1)

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
118287
5000
sample <class 'dict'>
dict_keys(['image', 'image/filename', 'image/id', 'objects'])
dict_keys(['area', 'bbox', 'id', 'is_crowd', 'label'])
sample <class 'dict'>
dict_keys(['image', 'image/filename', 'image/id', 'objects'])
dict_keys(['area', 'bbox', 'id', 'is_crowd', 'label'])


In [11]:
# list(train_dataset.take(1))

In [12]:
# list(train_loader.take(1))

In [13]:
# autotune = tf.data.experimental.AUTOTUNE
# train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=autotune)
# #     train_dataset = train_dataset.shuffle(8 * batch_size)
# train_dataset = train_dataset.shuffle(100)
# train_dataset = train_dataset.padded_batch(
#     batch_size=batch_size, padding_values=(0.0, 1e-8, -1), drop_remainder=True
# )
# train_dataset = train_dataset.map(
#     label_encoder.encode_batch, num_parallel_calls=autotune
# )
# train_dataset = train_dataset.apply(tf.data.experimental.ignore_errors())

In [14]:

# list(train_dataset.take(2))

In [15]:
# for image, label in train_loader:
#     print(image.shape)
#     print(label.shape)
# #     print(label)
#     outputs = model(image)
#     print(outputs.shape)
#     loss = loss_fn(label, outputs)
#     print(loss)    
    

In [ ]:
# Uncomment the following lines, when training on full dataset
# train_steps_per_epoch = dataset_info.splits["train"].num_examples // batch_size
# val_steps_per_epoch = \
#     dataset_info.splits["validation"].num_examples // batch_size
# train_steps = 4 * 100000
# epochs = train_steps // train_steps_per_epoch

epochs = 30

# Running 100 training and 50 validation steps,
# remove `.take` when training on the full dataset

model.fit(
#     train_dataset.take(100),
    train_loader.take(500),
    validation_data=val_loader.take(50),
    epochs=epochs,
    callbacks=callbacks_list,

)

Epoch 1/30
361/500 [====================>.........] - ETA: 35s - loss: 1.8330

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
# !pip install widgetsnbextension

In [9]:
# Change this to `model_dir` when not using the downloaded weights
#weights_dir = "data"
weights_dir = "retinanet"

latest_checkpoint = tf.train.latest_checkpoint(weights_dir)
model.load_weights(latest_checkpoint)

In [16]:
image = tf.keras.Input(shape=[None, None, 3], name="image")
predictions = model(image, training=False)
detections = DecodePredictions(confidence_threshold=0.5)(image, predictions)
inference_model = tf.keras.Model(inputs=image, outputs=detections)

In [21]:
def prepare_image(image):
    image, _, ratio = resize_and_pad_image(image, jitter=None)
    image = tf.keras.applications.resnet.preprocess_input(image)
    return tf.expand_dims(image, axis=0), ratio

val_dataset = tfds.load("coco/2017", split="validation", data_dir="data")
print('val dataset donwload complete')
int2str = dataset_info.features["objects"]["label"].int2str


val dataset donwload complete


In [19]:
type(val_dataset)
# list(val_dataset.take(1))

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [24]:
for sample in val_dataset.take(5):
    print(type(sample))
    image = tf.cast(sample["image"], dtype=tf.float32)
    input_image, ratio = prepare_image(image)
    print(input_image.shape)
    detections = inference_model.predict(input_image)
    num_detections = detections.valid_detections[0]
    print(num_detections)
    class_names = [
        int2str(int(x)) for x in detections.nmsed_classes[0][:num_detections]
    ]
    visualize_detections(
        image,
        detections.nmsed_boxes[0][:num_detections] / ratio,
        class_names,
        detections.nmsed_scores[0][:num_detections],
    )

<class 'dict'>
(1, 1152, 896, 3)


NotFoundError: 2 root error(s) found.
  (0) Not found:  No algorithm worked!
	 [[node model_1/RetinaNet/FeaturePyramid/model/conv1_conv/Conv2D (defined at /home/beomgon/Object_Detection/tf-retina/mobile-retina/notebooks/../model.py:55) ]]
	 [[model_1/decode_predictions/ExpandDims_2/_42]]
  (1) Not found:  No algorithm worked!
	 [[node model_1/RetinaNet/FeaturePyramid/model/conv1_conv/Conv2D (defined at /home/beomgon/Object_Detection/tf-retina/mobile-retina/notebooks/../model.py:55) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_predict_function_9171]

Function call stack:
predict_function -> predict_function
